In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from glob import glob

In [2]:
def open_cfg(file):
    with open(file, 'rt') as f:
        data = json.loads(f.read())

    for key in list(data.keys()):
        if type(data[key]) == list:
            data[key] = tuple(data[key])
    return data


In [3]:
results = sorted(glob('results/*.csv'))
print('N files:', len(results))
results = {log: pd.read_csv(log) for log in results}
print('N csvs open:', len(results))
results = {key.removeprefix('results/').removesuffix('.csv'): val for key, val in results.items()}
#print({key: len(val) for key, val in results.items() if not os.path.isfile(f'results/{key}.cfg')})
results = {key: {'log': val, 'cfg': open_cfg(f'results/{key}.cfg')} for key, val in results.items() if os.path.isfile(f'results/{key}.cfg')}
print('N csvs with cfgs:', len(results))
results = {key: val for key, val in results.items() if not key.endswith('pretrain')}
print('N cfgs that are not pretrain:', len(results))
results = {name: cfg['cfg'] for name, cfg in results.items()}
results = pd.DataFrame(results).T

N files: 108
N csvs open: 108
N csvs with cfgs: 108
N cfgs that are not pretrain: 96


In [4]:
results['name'] = results['name'].apply(lambda x: x.split('_')[0])

In [5]:
metrics_data = list()
metrics_in_question = 'mean_rocaucs'
main_cols = ['name', 'valid_fold', 'text_encoder_model', 'ecg_encoder_model', 'train_datasets', 'pretrained']
for index, row in results.iterrows():
    metrics = dict()
    for col in main_cols:
        metrics[col] = row[col]

    for col in ['test_metrics', 'zero_shot_test_metrics', 'exp2_metrics_trained', 'exp2_metrics_untrained']:
        for ds_name, ds_metrics in row[col].items():
            for val_name, val in ds_metrics.items():
                if val_name == metrics_in_question:
                    metrics[f'metrics__{col}__{ds_name}__{val_name}'] = val
            
    metrics_data.append(metrics)

In [6]:
metrics_data = pd.DataFrame(metrics_data)
metrics_cols = [col for col in metrics_data.columns if col.startswith('metrics__')]

In [7]:
metrics_data

,name,valid_fold,text_encoder_model,ecg_encoder_model,train_datasets,pretrained,metrics__test_metrics__ptb_xl__mean_rocaucs,metrics__test_metrics__ningbo__mean_rocaucs,metrics__test_metrics__georgia__mean_rocaucs,metrics__zero_shot_test_metrics__ptb_xl__mean_rocaucs,metrics__zero_shot_test_metrics__ningbo__mean_rocaucs,metrics__zero_shot_test_metrics__georgia__mean_rocaucs,metrics__exp2_metrics_trained__sph__mean_rocaucs,metrics__exp2_metrics_trained__code15__mean_rocaucs,metrics__exp2_metrics_untrained__sph__mean_rocaucs,metrics__exp2_metrics_untrained__code15__mean_rocaucs
0,04728b1accd4,1,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo, georgia)",False,0.771608,0.793529,0.714515,0.668172,0.663193,0.688492,0.843566,None,0.705168,0.651023
1,04728b1accd4,2,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo, georgia)",False,0.731239,0.808144,0.700091,0.636798,0.655746,0.695727,0.827527,None,0.707302,0.612331
2,04728b1accd4,3,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo, georgia)",False,0.741044,0.788158,0.706854,0.619278,0.649107,0.693829,0.808446,None,0.670876,0.657726
3,04728b1accd4,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo, georgia)",False,0.703586,0.776901,0.695009,0.627984,0.677610,0.699854,0.788665,None,0.646339,0.613014
4,133dd7f69cdb,1,emilyalsentzer/Bio_ClinicalBERT,RNN_model,"(ptb_xl, ningbo)",False,0.700645,0.741786,NaN,0.621800,0.633456,NaN,0.782823,None,0.646379,0.621253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,f21a8efbe7bb,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo)",False,0.727947,0.755018,NaN,0.616374,0.613793,NaN,0.822452,None,0.691401,0.611537
92,f98c24cbc6f4,1,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl, ningbo, georgia)",False,0.718761,0.771160,0.697617,0.624308,0.625716,0.692302,0.807074,None,0.708339,0.647052
93,f98c24cbc6f4,2,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl, ningbo, georgia)",False,0.729888,0.827820,0.725558,0.611287,0.657587,0.703056,0.834860,None,0.683702,0.648375
94,f98c24cbc6f4,3,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl, ningbo, georgia)",False,0.717661,0.778368,0.694815,0.569470,0.630734,0.707919,0.805691,None,0.704377,0.595756


In [8]:
groups = metrics_data.groupby('name')[metrics_cols].agg(['mean', 'std'])

In [9]:
groups

metrics__test_metrics__ptb_xl__mean_rocaucs            \
                                                    mean       std   
name                                                                 
04728b1accd4                                    0.736870  0.028070   
133dd7f69cdb                                    0.714149  0.010703   
19d8db9da7c9                                    0.683529  0.009770   
1daca59f89f0                                    0.710832  0.013714   
1df7bd5e6bc5                                    0.690341  0.006571   
29edb150bba1                                    0.730679  0.010309   
29efd10cc35b                                    0.696417  0.013248   
2b74245b2760                                    0.716478  0.010680   
30771647826f                                    0.706836  0.014548   
39a0aee494e6                                    0.739238  0.017618   
50cebc3f0fce                                    0.707103  0.039605   
52e1b29284b3                                    0.743339  0.015811   
68573c2b98c0                                    0.725126  0.015629   
69e4f4215133                                    0.702005  0.015133   
7748de44d79e                                    0.691578  0.011657   
7c2e8b6061c9                                    0.716710  0.015775   
7ebc188c6d5e                                    0.719603  0.008013   
9debdc44efdd                                    0.691051  0.016371   
9dfd6477a359                                    0.719206  0.014377   
cc4eb0111e23                                    0.705519  0.008683   
d91b15e34ce7                                    0.739216  0.008816   
dd9086d44551                                    0.693580  0.008690   
f21a8efbe7bb                                    0.719083  0.013407   
f98c24cbc6f4                                    0.729967  0.016668   

             metrics__test_metrics__ningbo__mean_rocaucs            \
                                                    mean       std   
name                                                                 
04728b1accd4                                    0.791683  0.012978   
133dd7f69cdb                                    0.711769  0.021917   
19d8db9da7c9                                         NaN       NaN   
1daca59f89f0                                         NaN       NaN   
1df7bd5e6bc5                                    0.707921  0.010880   
29edb150bba1                                    0.765246  0.028528   
29efd10cc35b                                         NaN       NaN   
2b74245b2760                                    0.725537  0.024389   
30771647826f                                    0.739059  0.022547   
39a0aee494e6                                    0.750640  0.008401   
50cebc3f0fce                                         NaN       NaN   
52e1b29284b3                                    0.757162  0.021546   
68573c2b98c0                                    0.760448  0.013300   
69e4f4215133                                         NaN       NaN   
7748de44d79e                                         NaN       NaN   
7c2e8b6061c9                                    0.714284  0.023223   
7ebc188c6d5e                                    0.733009  0.020584   
9debdc44efdd                                         NaN       NaN   
9dfd6477a359                                    0.741157  0.041531   
cc4eb0111e23                                    0.711128  0.023707   
d91b15e34ce7                                    0.784682  0.025209   
dd9086d44551                                         NaN       NaN   
f21a8efbe7bb                                    0.761082  0.022943   
f98c24cbc6f4                                    0.802377  0.032069   

             metrics__test_metrics__georgia__mean_rocaucs            \
                                                     mean       std   
name                                                                  
04728b1accd4      

In [10]:
results = list()
for name, row in groups.iterrows():
    config = metrics_data.loc[metrics_data['name'] == name, main_cols]
    #assert len(config) == 4
    print()
    results_row = dict()
    results_row['n_configs'] = len(config)
    print('Number of configs:', len(config))
    config = config.iloc[0].to_dict()
    
    results_row.update(config)
    
    
    for key, val in config.items():
        if key != 'valid_fold':
            print(key, ':', val)
    print()
    for name, val in row.items():
        if name[1] == 'mean':
            mean_val = val
        elif name[1] == 'std':
            
            metrics_type = name[0].split("__")[1]
            dataset = name[0].split("__")[2]
            val_string = f'{mean_val:.3f}+/-{val:.3f}'
            
            print(f'{metrics_type}: {dataset} Mean ROC-AUCS = {val_string}')
            results_row[f'{metrics_type}__{dataset}'] = f'{val_string}'
    results.append(results_row)


Number of configs: 4
name : 04728b1accd4
text_encoder_model : emilyalsentzer/Bio_ClinicalBERT
ecg_encoder_model : CNN_model_v3
train_datasets : ('ptb_xl', 'ningbo', 'georgia')
pretrained : False

test_metrics: ptb_xl Mean ROC-AUCS = 0.737+/-0.028
test_metrics: ningbo Mean ROC-AUCS = 0.792+/-0.013
test_metrics: georgia Mean ROC-AUCS = 0.704+/-0.008
zero_shot_test_metrics: ptb_xl Mean ROC-AUCS = 0.638+/-0.021
zero_shot_test_metrics: ningbo Mean ROC-AUCS = 0.661+/-0.012
zero_shot_test_metrics: georgia Mean ROC-AUCS = 0.694+/-0.005
exp2_metrics_trained: sph Mean ROC-AUCS = 0.817+/-0.024
exp2_metrics_trained: code15 Mean ROC-AUCS = nan+/-nan
exp2_metrics_untrained: sph Mean ROC-AUCS = 0.682+/-0.029
exp2_metrics_untrained: code15 Mean ROC-AUCS = 0.634+/-0.024

Number of configs: 4
name : 133dd7f69cdb
text_encoder_model : emilyalsentzer/Bio_ClinicalBERT
ecg_encoder_model : RNN_model
train_datasets : ('ptb_xl', 'ningbo')
pretrained : False

test_metrics: ptb_xl Mean ROC-AUCS = 0.714+/-0.011
t

In [11]:
results = pd.DataFrame(results)
del results['valid_fold']
del results['name']
results = results.sort_values(['ecg_encoder_model', 'text_encoder_model', 'train_datasets',  'pretrained'])

In [12]:
results.to_csv('docs/results.csv')

In [13]:
results

,n_configs,text_encoder_model,ecg_encoder_model,train_datasets,pretrained,test_metrics__ptb_xl,test_metrics__ningbo,test_metrics__georgia,zero_shot_test_metrics__ptb_xl,zero_shot_test_metrics__ningbo,zero_shot_test_metrics__georgia,exp2_metrics_trained__sph,exp2_metrics_trained__code15,exp2_metrics_untrained__sph,exp2_metrics_untrained__code15
17,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model,"(ptb_xl,)",False,0.691+/-0.016,nan+/-nan,nan+/-nan,0.563+/-0.037,nan+/-nan,nan+/-nan,0.698+/-0.034,nan+/-nan,0.568+/-0.043,0.578+/-0.042
7,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model,"(ptb_xl, ningbo)",False,0.716+/-0.011,0.726+/-0.024,nan+/-nan,0.613+/-0.017,0.641+/-0.014,nan+/-nan,0.786+/-0.028,nan+/-nan,0.666+/-0.019,0.623+/-0.017
20,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model,"(ptb_xl, ningbo, georgia)",False,0.739+/-0.009,0.785+/-0.025,0.701+/-0.024,0.632+/-0.024,0.668+/-0.032,0.698+/-0.016,0.817+/-0.016,nan+/-nan,0.665+/-0.028,0.636+/-0.019
3,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl,)",False,0.711+/-0.014,nan+/-nan,nan+/-nan,0.541+/-0.021,nan+/-nan,nan+/-nan,0.717+/-0.038,nan+/-nan,0.577+/-0.025,0.540+/-0.024
15,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl, ningbo)",False,0.717+/-0.016,0.714+/-0.023,nan+/-nan,0.610+/-0.027,0.601+/-0.004,nan+/-nan,0.784+/-0.038,nan+/-nan,0.670+/-0.038,0.608+/-0.029
23,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v2,"(ptb_xl, ningbo, georgia)",False,0.730+/-0.017,0.802+/-0.032,0.710+/-0.016,0.615+/-0.035,0.640+/-0.015,0.698+/-0.009,0.829+/-0.029,nan+/-nan,0.703+/-0.014,0.637+/-0.028
10,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl,)",False,0.707+/-0.040,nan+/-nan,nan+/-nan,0.544+/-0.059,nan+/-nan,nan+/-nan,0.703+/-0.081,nan+/-nan,0.585+/-0.091,0.579+/-0.064
22,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo)",False,0.719+/-0.013,0.761+/-0.023,nan+/-nan,0.599+/-0.017,0.646+/-0.023,nan+/-nan,0.805+/-0.027,nan+/-nan,0.668+/-0.019,0.606+/-0.034
0,4,emilyalsentzer/Bio_ClinicalBERT,CNN_model_v3,"(ptb_xl, ningbo, georgia)",False,0.737+/-0.028,0.792+/-0.013,0.704+/-0.008,0.638+/-0.021,0.661+/-0.012,0.694+/-0.005,0.817+/-0.024,nan+/-nan,0.682+/-0.029,0.634+/-0.024
6,4,dmis-lab/biobert-v1.1,ISIBrno_model,"(ptb_xl,)",False,0.696+/-0.013,nan+/-nan,nan+/-nan,0.596+/-0.019,nan+/-nan,nan+/-nan,0.721+/-0.039,nan+/-nan,0.593+/-0.043,0.580+/-0.022
